# Federated learning: regression using the California Housing database

In this notebook we are going to show how you can use a federated learning environment to create a regression model. 
In the notebook ["Federated learning: regression for a 2D simple case"](./federated_learning_regression_simple.ipynb) we explained the basic concepts of the framework, so now we will go slightly faster.

First of all, we load a dataset (included in the framework) to allow for regression experiments.

In [1]:
import shfl
from shfl.data_base.california_housing import CaliforniaHousing

database = CaliforniaHousing()
train_data, train_labels, val_data, val_labels, test_data, test_labels = database.load_data()

Using TensorFlow backend.


We are going to explore the data

In [2]:
print("Shape of train_data: " + str(train_data.shape))
print("Shape of train_labels: " + str(train_labels.shape))
print("One sample features: " + str(train_data[0]))
print("One sample label: " + str(train_labels[0]))

Shape of train_data: (16512, 8)
Shape of train_labels: (16512,)
One sample features: [ 3.39060000e+00  1.50000000e+01  4.02361111e+00  1.01111111e+00
  2.34000000e+03  3.25000000e+00  3.76700000e+01 -1.22480000e+02]
One sample label: 3.037


Federeted data generation:

In [3]:
import shfl

iid_distribution = shfl.data_distribution.IidDataDistribution(database)
federated_data, test_data, test_label = iid_distribution.get_federated_data(20, percent=10)

Model definition:

In [4]:
import keras

def model_builder():
    # create model
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=["mae"])
    
    return shfl.model.DeepLearningModel(model)

Federated environment definition:

In [5]:
aggregator = shfl.federated_aggregator.AvgFedAggregator()
federated_government = shfl.learning_approach.FederatedGovernment(model_builder, federated_data, aggregator)

Reshaping data:

In [6]:
import numpy as np

class Reshape(shfl.private.FederatedTransformation):
    
    def apply(self, labeled_data):
        labeled_data.label = np.reshape(labeled_data.label, (labeled_data.label.shape[0], 1))
        
shfl.private.federated_operation.apply_federated_transformation(federated_data, Reshape())

Running experiment:

In [7]:
test_label = np.reshape(test_label, (test_label.shape[0], 1))
federated_government.run_rounds(3, test_data, test_label)

Accuracy round 0
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x13d9d6410>: [6.30829996101616, 1.6697971820831299]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x13d9d6d50>: [4.9624294051828315, 1.8432769775390625]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x13d9d6dd0>: [4.478764112605605, 1.5718355178833008]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x13d9d6ad0>: [4.735966440319091, 1.5490785837173462]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x13d9d6850>: [4.3795931875243665, 1.5281556844711304]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x13d9d6e90>: [4.479200812273247, 1.5332965850830078]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x13d9d6f10>: [5.015918417494427, 1.855161190032959]
Tes